# **Trabajo Práctico 1: Reservas de Hotel**
## Checkpoint 2
**Grupo 35**

Integrantes:
- Aylas, Brenda
- Cori, William
- Nazario, Ingrith

In [1]:
import dill
import numpy as np
import pandas as pd
from category_encoders import BinaryEncoder, OneHotEncoder
from datetime import datetime
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from pathlib import Path

pd.set_option('display.max_columns', None)

### Carga de datos

Cargo el dataset de train procesado en el primer checkpoint y lo separo en X e Y.
Tambien cargo el dataset de test y separo los ids para unirlos luego a los valores que prediga el modelo.

In [2]:
hotels_train = pd.read_csv("../data/hotels_processed.csv")

x_train = hotels_train.drop("is_canceled", axis=1)
y_train = hotels_train.filter(["is_canceled"])

x_test = pd.read_csv("../data/hotels_test.csv")
ids_test = x_test.loc[:,"id"]


### Armado de pipeline

Armo un pipeline en el que utilizo one hot encoding para las columnas categóricas con pocas categorias distintas, utilizo binary enconding para las columnas con muchas categorias distintas (>12) para no generar una excesiva cantidad de columnas y descarto las columnas que se decidio elimnar en el primer checkpoint o son prohibidas para este trabajo practico. El modelo utilizado es DecisionTreeClassifier de Scikitlearn.

In [3]:
one_hot_encoder_cols = [
  'hotel',
  'arrival_date_month',
  'meal',
  'market_segment',
  'distribution_channel',
  'reserved_room_type',
  'assigned_room_type',
  'deposit_type',
  'customer_type',
  'is_repeated_guest'
]

binary_encoder_cols = [
  'country',
  'agent'
]

ignored_cols = [
  "reservation_status_date",
  "id",
  "company",
]

def drop_colums(X: pd.DataFrame):
  return [col for col in X.columns if col in ignored_cols]

column_trans = ColumnTransformer(
  transformers=(
    ("binary_encoder", BinaryEncoder(cols=binary_encoder_cols), binary_encoder_cols),
    ("one_hot_encoder", OneHotEncoder(cols=one_hot_encoder_cols), one_hot_encoder_cols),
    ("drop", "drop", drop_colums)
  ),
  remainder="passthrough"
)

pipeline = Pipeline(
  steps=[
    ("transformer", column_trans),
    ("model", DecisionTreeClassifier())
  ])


### Busqueda de hiperparametros

Utilizo GridSearchCV de scikitlearn para probar distintos valores de hiperparametros. Para ejecutarlo hay que setear el boolean en la celda(Tiempo de ejecucion aproximado: 20min)

In [4]:
params = {
  'model__criterion':['gini', 'entropy'],
  'model__min_samples_leaf':[3, 5, 10, 15, 20, 25],
  'model__ccp_alpha':np.linspace(0,0.05, 20),
  'model__max_depth':list(range(20, 51, 5))
}

grid_search = GridSearchCV(
  estimator=pipeline,
  param_grid=params,
  scoring="f1",
  n_jobs=-1,
  cv=5
)

grid_search_enable = False
if grid_search_enable:
  grid_search.fit(x_train, y_train)

  print(f"Best params: {grid_search.best_params_}")
  print(f"F1 score: {grid_search.best_score_}")

Entreno el modelo con los hiperparametros finales, lo uso con el dataset de test y armo la prediccion a presentar en kaggle

In [5]:
best_params = grid_search.best_params_ if grid_search_enable else {'model__ccp_alpha': 0.0, 'model__criterion': 'entropy', 'model__max_depth': 20, 'model__min_samples_leaf': 15}
pipeline.set_params(**best_params)
pipeline.fit(x_train, y_train)

y_predict = pipeline.predict(x_test)
result = pd.DataFrame({"id": ids_test, "is_canceled": y_predict})

### Guardo el modelo y el resultado a presentar

In [6]:
tmp = "../tmp"
Path(tmp).mkdir(exist_ok=True)
now = datetime.now().strftime("%Y_%m_%dd_%H_%M_%S")

result.to_csv(f"{tmp}/result_{now}.csv")
with open(f"{tmp}/decision_tree_classifier_{now}", "wb") as fh:
  dill.dump(pipeline, fh)